In [1]:
import os
gpu_num = 0 # 使用 "" 来启用 CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
import mitsuba as mi
import open3d as o3d
from open3d.web_visualizer import draw

# Import Sionna RT components
from mysionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera
from mysionna.rt.scattering_pattern import *
from sionna.utils import insert_dims

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [2]:
scene = load_scene("Indoor/indoor.xml")

In [3]:
#################配置发端天线阵列#################
scene.tx_array = PlanarArray(num_rows=2,
                             num_cols=2,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="dipole",
                             polarization="H")

#################配置收端天线阵列#################
scene.rx_array = PlanarArray(num_rows=2,
                             num_cols=2,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="dipole",
                             polarization="H")

################创建发射机#########################
tx = Transmitter(name="tx",
                 position=[0,0,2.95])
################ 将发射机加入到场景中##############
scene.add(tx)
#################创建接收机########################
rx = Receiver(name="rx",
              position=[0,0,2.95])
################ 将接收机加入到场景中##############
scene.add(rx)
tx.look_at([0,0,0])
rx.look_at([0,0,0])

In [4]:
scene.frequency = 2.14e9 # in Hz; implicitly updates RadioMaterials
scene.synthetic_array = False # If set to False, ray tracing will be done per antenna element (slower for large arrays)

In [5]:
scene.target_names = ["door"]
scene.target_velocities = [(5.,5.,0.)]

In [6]:
p1 = LambertianPattern()
p2 = DirectivePattern(50)
scene.get("itu_plywood").scattering_coefficient = 0.4
scene.get("itu_plywood").scattering_pattern = p2
scene.get("itu_concrete").scattering_coefficient = 0.7
scene.get("itu_concrete").scattering_pattern = p1
scene.get("itu_floorboard").scattering_coefficient = 0.3
scene.get("itu_floorboard").scattering_pattern = p2
scene.get("itu_ceiling_board").scattering_coefficient = 0.7
scene.get("itu_ceiling_board").scattering_pattern = p1

In [7]:
paths = scene.compute_paths(max_depth=4,diffraction=True,scattering=True,edge_diffraction=True,scat_keep_prob=0.001)

In [8]:
# scene.preview(paths=paths)

In [9]:
v,obj=scene.compute_target_velocities(paths, return_obj_names=True)

In [10]:
subcarrier_spacing = 15e3
num_time_steps = 14
paths.apply_doppler(sampling_frequency=subcarrier_spacing, num_time_steps=num_time_steps,target_velocities=v)

In [11]:
a,tau = paths.cir()
mask = paths.mask
del paths

In [13]:
frequency = scene.frequency
length = a.shape[5]
snr=10
phase = tf.complex(tf.zeros_like(tau),2*np.pi*frequency*tau)
e = tf.exp(phase)
e = tf.expand_dims(e, axis=-1)
a = a * e
tau_i = tf.repeat(tau,length,axis=-1)
tau_i = tf.reshape(tau_i, [tau.shape[0],tau.shape[1],tau.shape[2],tau.shape[3],tau.shape[4],length,length])
tau_j = tf.transpose(tau_i,perm=[0,1,2,3,4,6,5])
tau_i_mine_j = tau_i- tau_j
tau_i_mul_j = tau_i* tau_j
tau_i_mine_j = tf.expand_dims(tau_i_mine_j, axis=-1)
tau_i_mul_j = tf.expand_dims(tau_i_mul_j, axis=-1)
alpha = tf.expand_dims(a, axis=-2)
alpha_1 = tf.transpose(alpha,perm=[0,1,2,3,4,7,5,6])
alpha_2 = tf.transpose(alpha,perm=[0,1,2,3,4,7,6,5])
alpha_ij = tf.matmul(alpha_1,alpha_2)
alpha_ij = tf.transpose(alpha_ij,perm=[0,1,2,3,4,6,7,5])
one = tf.ones((length,length))
one = insert_dims(one, 5, 0)
one = insert_dims(one, 1, -1)
F_alpha= 2*snr*tf.math.abs(alpha_ij)/(tau_i_mul_j**2)
F_cos = (one+4*(np.pi**2)*(frequency) * tau_i_mul_j)*tf.math.cos(2*np.pi*frequency*tau_i_mine_j)
F_sin = 2*np.pi*frequency*tau_i_mine_j*tf.math.sin(2*np.pi*frequency*tau_i_mine_j)
F = F_alpha*(F_cos+F_sin)
del alpha_1,alpha_2,alpha_ij,tau_i_mine_j,tau_i_mul_j,tau_i,tau_j,F_alpha,F_cos,F_sin
F = F*10e-18

ResourceExhaustedError: {{function_node __wrapped__BatchMatMulV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[1,1,4,1,4,14,1198,1198] and type complex64 on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:BatchMatMulV2] name: 

In [ ]:
crb_F = tf.linalg.pinv(F)
crb = tf.linalg.diag_part(crb_F)
crb = tf.math.abs(crb)*10e-18

In [23]:
tf.math.abs(tf.linalg.diag_part(tf.linalg.pinv(F[0,0,0,0,0,:,:,0])))*10e-18

<tf.Tensor: shape=(1199,), dtype=float32, numpy=
array([0.0000000e+00, 0.0000000e+00, 3.6859726e-36, ..., 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00], dtype=float32)>

In [13]:
objects = paths.objects
indices = tf.where(objects!=-1)
vertices = paths.vertices
v = tf.gather_nd(vertices, indices)


In [24]:
v = tf.reshape(v,[-1,3])
v = v.numpy()
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(v)
pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0,1,[v.shape[0],3]))
o3d.io.write_point_cloud("test.xyzrgb", pcd)

True